In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

file_url = '../datasets/cnnhealth.txt'
csvfile = './mycsv.csv'

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# File url
file_url = '../datasets/cnnhealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
# -------------------------------------------------
frame = pd.read_table(file_url, sep='|', header= None, names = csv_header, encoding="utf-8")
frame.head()

,tweetid,date,title
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...


In [3]:
# with open(file_url, 'r') as infile, open(csvfile, 'w') as outfile:
#      stripped = (line.strip() for line in infile)
#      lines = (line.split("|") for line in stripped if line)
#      writer = csv.writer(outfile)
#      writer.writerows(lines)

In [4]:
# -------------------------------------------------
frame.shape

(4057, 3)

In [5]:
# Merge column 2 & 3
def join_txt(list_txt):
    # Join list of content into 1 simple paragraph
    seperator = ', '
    return seperator.join(list_txt)

In [6]:
# -------------------------------------------------
frame.loc[:,'title'][0]

'An abundance of online info can turn us into e-hypochondriacs. Or, worse, lead us to neglect getting the care we need http://cnn.it/1L1t1Fv'

In [7]:
# -------------------------------------------------
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(split_txt_form_url)

http://cnn.it/1L1t1Fv
http://cnn.it/1xdpsjT
http://cnn.it/1BOphBk
http://cnn.it/1GNdmqc
http://cnn.it/1xls8w5
http://cnn.it/1GNiH0L
http://cnn.it/1AklXJQ
http://cnn.it/1AklXJQ
http://cnn.it/1GNhBC8
http://cnn.it/18HDKDg
http://cnn.it/1EhlrjJ
http://cnn.it/1AukT5d
http://cnn.it/1x1ffMU
http://cnn.it/18GEfgH
http://cnn.it/1Gxrm3R
http://cnnmon.ie/1Eh7Usn
http://cnn.it/1MuzOTx
http://cnn.it/1BxKgIa.
http://cnn.it/1Gvs95n
http://cnnmon.ie/1GKz7qP
http://cnn.it/1CbnIPC
http://ti.me/1Gvz26A
http://cnn.it/1xg6t8x
http://cnn.it/1L1t1Fv
http://bitly.com/1B7WimB
http://cnn.it/1FfcSYb
http://cnn.it/1Mvai0w
http://cnn.it/1EfbKSL
http://cnn.it/1GIzkux
http://cnn.it/1Ak1hAE
http://cnn.it/1AhuRI7
http://cnn.it/1AhvHnW
http://cnn.it/1L1t1Fv
http://cnnmon.ie/1HEyDAd
http://cnn.it/1HEjTRR
http://cnn.it/1xdpsjT
http://cnn.it/1C6vc6z
http://cnn.it/1xdpsjT
http://bit.ly/1ymrovI
http://cnn.it/1C6vc6z
http://cnnmon.ie/1ESxFR5
http://cnn.it/1MrpygJ
http://cnnmon.ie/18xMYBU
http://cnnmon.ie/1C4SEB4
http://cnn.

http://cnn.it/17MyFmK
http://cnn.it/17K4NMQ
http://cnn.it/1aOXyxO
http://cnn.it/1aC2JEJ
http://cnn.it/1aC2JEJ
http://cnn.it/1ioylSD
http://cnn.it/18IVmYU
http://cnn.it/1fOA5SS
http://cnn.it/1j4zZ9B
http://cnn.it/1iZSImq
http://cnn.it/1e3yIPx
http://cnn.it/1j98j3j
http://cnn.it/HUpRoe
http://cnn.it/1j98j3j
http://cnn.it/HUpRoe
http://www.cnn.com/2013/11/14/health/dogs-domesticated-europe/
http://cnn.it/1j97JCK
http://ow.ly/qPkkG
http://cnn.it/1aG55z0
http://cnn.it/1dogt6D
http://cnn.it/1aVbIUi
http://t.co/7a74p
http://cnn.it/HRDl4h
http://cnn.it/18s6rO8
http://cnn.it/1e3yIPx
http://cnn.it/1cUQpUi
http://cnn.it/1j4zZ9B
http://cnn.it/1brU9cE
http://cnn.it/16YXBKi
http://cnn.it/HV4jas
http://cnn.it/181UthI
http://cnn.it/1iZSImq
http://www.cnn.com/2013/11/11/health/typhoon-medical-crisis/index.html?sr
http://cnn.it/1buBifa
http://cnn.it/17PB3Pa
http://cnn.it/1i0VTww
http://cnn.it/1bnLCaE
http://cnn.it/1bdm3sW
http://cnn.it/195BTRv
http://cnn.it/1bdm3sW
http://cnn.it/1bdm3sW
http://cnn.it/1h

http://at.cnn.com/497aRfP
http://at.cnn.com/fJqoPg7
http://at.cnn.com/v2XWR3Z
http://at.cnn.com/5hlUBHC
http://at.cnn.com/EkpVRlI
http://at.cnn.com/r7oiLaU
http://at.cnn.com/jiGVDXE
http://at.cnn.com/7qYmE3s
http://at.cnn.com/uUz3quA
http://at.cnn.com/iMDXH39
http://at.cnn.com/Vi5LmHQ
http://at.cnn.com/qeih0Pe
http://at.cnn.com/zWuHTvu
http://at.cnn.com/vMfxbCj
http://at.cnn.com/pqKlcCX
http://at.cnn.com/HHoXmNE
http://at.cnn.com/TyJOVgb
http://at.cnn.com/YqPpczo
http://at.cnn.com/gQUyvox
http://at.cnn.com/xlCnV26
http://at.cnn.com/OW43lsz
http://at.cnn.com/ajqRHWG
http://at.cnn.com/0nQ3RYe
http://at.cnn.com/ygqcdQW
http://at.cnn.com/3jefspI
http://at.cnn.com/CMJ62kQ
http://at.cnn.com/8ldgNAq
http://at.cnn.com/n4tsKvW
http://at.cnn.com/pnxHPeF
http://at.cnn.com/wsfyjiF
http://at.cnn.com/4iKDNot
http://at.cnn.com/tmwsQJN
http://at.cnn.com/Fz8ByS5
http://at.cnn.com/008RelO
http://at.cnn.com/PgOvCgx
http://at.cnn.com/8p70nHu
http://at.cnn.com/4XVxbhi
http://at.cnn.com/yONIkjw
http://at.cn



http://cnn.it/1C2Fobs
It's hard to believe it's 5 years since the Haiti quake. Here's one incredible survival story http://cnn.it/14PwiUP 


http://cnn.it/14PwiUP
A 'one in a million' chance, that's what this couple had for having these babies http://cnn.it/1ybMEBC 


http://cnn.it/1ybMEBC
5 years ago an infant was left alone buried in the rubble for 4 entire days.Check her out now http://cnn.it/14PwiUP 


http://cnn.it/14PwiUP
More are sick &amp; federal investigators may know what's causing it with some tainted apples http://cnn.it/1AQgM7E 


http://cnn.it/1AQgM7E
Teen's forced chemo may continue, Connecticut court rules http://cnn.it/14wifnI 


http://cnn.it/14wifnI
By day's end alcohol poisoning will have killed 6 people. Here's why  @jechristensen http://cnn.it/14uwUje 


http://cnn.it/14uwUje
The impossible becomes possible: technology helps a paralyzed man move his legs @elizcohencnn http://cnn.it/1DeH1qE 


http://cnn.it/1DeH1qE
RT @drsanjaygupta: stuck at the office? you're 


http://cnn.it/1kef9Fz
RT @djblatner: #Exercise alone isn't enough to lose weight @CNNHealth.Myths about #weightoss from @retrofitme Advisor Dr Brad Saks: http://


Think you really can OD from #caffeine? Here's how it works http://www.cnn.com/2013/12/03/health/upwave-caffeine-overdose/index.html?hpt=he_c2


http://www.cnn.com/2013/12/03/health/upwave-caffeine-overdose/index.html?hpt
Find out how men's brains change in response to fatherhood.  http://edition.cnn.com/2014/07/01/health/fathers-brain-changes/


http://edition.cnn.com/2014/07/01/health/fathers-brain-changes/
Is this the end of the debate? Exhaustive research finds vaccines for kids are safe. Really. from @jechristensen http://www.cnn.com/2014/07/01/health/vaccines-for-kids-safe/index.html?hpt=hp_inthenews 


http://www.cnn.com/2014/07/01/health/vaccines-for-kids-safe/index.html?hpt
What your next gyno exam may skip http://cnn.it/1qbWL5R


http://cnn.it/1qbWL5R
1/3 of knee replacements are not "appropriate," a new study fin

...

Is hand sanitizer toxic? http://www.cnn.com/2013/10/16/health/hand-sanitizer-toxic-upwave/index.html via @upwave #cold #flu #germs


http://www.cnn.com/2013/10/16/health/hand-sanitizer-toxic-upwave/index.html
Flesh-eating zombie drug Krokodil 'kills you from the inside out' http://www.cnn.com/2013/10/16/health/krokodil-zombie-drug/index.html


http://www.cnn.com/2013/10/16/health/krokodil-zombie-drug/index.html
You'll be speaking at ONA Thursday? @tiffanycampbell Looking forward to seeing you there!


Brain scientists getting closer to decoding thoughts http://on.cnn.com/1bwg1Co


http://on.cnn.com/1bwg1Co
30,000 in UK may be carriers of human form of mad cow disease http://thechart.blogs.cnn.com/2013/10/15/30000-may-carry-human-form-of-mad-cow/


http://thechart.blogs.cnn.com/2013/10/15/30000-may-carry-human-form-of-mad-cow/
BPA, phthalates linked to fertility problems and miscarriages in new research from @ReprodMed #IFFSASRM http://www.cnn.com/2013/10/15/health/bpa-miscarriage-phtha

http://at.cnn.com/4zI2vnq
Mark Harmon from @NCIS_CBS is in a new @StandUpToCancer &amp; @CancerResearchInstitute PSA. See it here! http://at.cnn.com/8PNbinM


http://at.cnn.com/8PNbinM
Surgeon general stepping down http://at.cnn.com/rKqTdi2


http://at.cnn.com/rKqTdi2
BPA linked to obesity AGAIN -- this time in young girls http://at.cnn.com/KrSnToR


http://at.cnn.com/KrSnToR
Pain in the pumps? Why some women choose couture over comfort http://at.cnn.com/jmjW0n9


http://at.cnn.com/jmjW0n9
SCARY: 98% of childhood #cancer survivors have at least one chronic disease http://at.cnn.com/6IpcX


http://at.cnn.com/6IpcX
Dr. Phil: Protect yourself from BAITERs http://at.cnn.com/hn4nc7S


http://at.cnn.com/hn4nc7S
Plan B for all http://at.cnn.com/kwRpic2


http://at.cnn.com/kwRpic2
5 things to know about carbon monoxide http://at.cnn.com/10lcOif


http://at.cnn.com/10lcOif
Healthy Father's Day gadgets for Dad http://at.cnn.com/KvSjEt1


http://at.cnn.com/KvSjEt1
New weight-loss drug available h

http://at.cnn.com/IQvOQmo
Woman running 1 marathon every week http://at.cnn.com/hb42hzR


http://at.cnn.com/hb42hzR
What did you learn from bodybuilding pioneer Joe Weider? Tell us with #LessonsfromJoe http://at.cnn.com/LnGshJD


http://at.cnn.com/LnGshJD
Darkness descends over returning vets http://at.cnn.com/IXPVsdo


http://at.cnn.com/IXPVsdo
Today's #getfit tip @MensHealth: Working the night shift? Adjust your concept of mealtime &amp; eat the biggest meal when you get home


Norovirus leading cause of intestinal disorders in kids http://at.cnn.com/QvF55lf


http://at.cnn.com/QvF55lf
Why being part of a team matters http://at.cnn.com/nR5vqHh #triwithme (via @TriHardDouglas)


http://at.cnn.com/nR5vqHh
Toddler meals swimming in sodium http://at.cnn.com/d1rxMzl


http://at.cnn.com/d1rxMzl
Today's #getfit tip @MsMerOram: Call it by a different name. My eggplant veggie lasagna is "pizza in a cup" to my 3 yr old #kids


Couple loses 500 pounds in two years http://at.cnn.com/G3I11HP #wei


http://at.cnn.com/TtUGD54
Loneliness linked to dementia http://at.cnn.com/OrxGoyt


http://at.cnn.com/OrxGoyt
In-laws can help -- or hurt -- your marriage http://at.cnn.com/Li99Oh6


http://at.cnn.com/Li99Oh6
California Gov. Jerry Brown treated for prostate cancer http://at.cnn.com/RSoCfRt


http://at.cnn.com/RSoCfRt
Doctors urged: keep kids from smoking http://at.cnn.com/AEjV3nh


http://at.cnn.com/AEjV3nh
A different holiday trip: The guilt trip http://at.cnn.com/uFJ3LnD


http://at.cnn.com/uFJ3LnD
Another Sandy legacy: Home mold http://at.cnn.com/WQNyEOf


http://at.cnn.com/WQNyEOf
Caregiver lives rerouted yet enriched by aging parents http://at.cnn.com/dKBjvPs


http://at.cnn.com/dKBjvPs
Give a heart-healthy gift http://at.cnn.com/rA2xbIK


http://at.cnn.com/rA2xbIK
Frankie Muniz: Mini-stroke was 'wake-up call' http://at.cnn.com/hMBCiav


http://at.cnn.com/hMBCiav
Opinion: Drug industry's free speech helps doctors http://at.cnn.com/YalSA7P


http://at.cnn.com/YalSA7P
HIV as cancer



http://at.cnn.com/jwwGsc
Doc says there's one glaring omission in 'Obamacare' http://at.cnn.com/S0d6YP #healthcare


http://at.cnn.com/S0d6YP
'Sex and the City' actress: Talk to your kids about sex http://at.cnn.com/xyM1No


http://at.cnn.com/xyM1No
Former Navy SEAL @GorillaFrog runs half marathons with someone else's lungs http://at.cnn.com/8GqpH3 #cancersurvivor


http://at.cnn.com/8GqpH3
Is 'secondhand' TV taking a toll on kids? http://at.cnn.com/O0aWMc


http://at.cnn.com/O0aWMc
#Health Minute: Tips for bathing a newborn http://at.cnn.com/TIHgjM


http://at.cnn.com/TIHgjM
Deadly snake venom could be the new pain medication http://at.cnn.com/FxmVK2


http://at.cnn.com/FxmVK2
Cancer patient fights parents for right to die http://at.cnn.com/CL1SvX


http://at.cnn.com/CL1SvX
Tech can reduce our stress, too @amandaenayati  says http://at.cnn.com/QcYQVH #ourmobilesociety


http://at.cnn.com/QcYQVH
Peanut, nut butter recall expanded http://at.cnn.com/NsAbvB


http://at.cnn.com/NsAbvB
Wh

In [8]:
# rows = []
# csv_header = ['tweetid','date','title']
# frame_header = ['tweetid','date','title','what']

# with open(file_url, 'rt', encoding='utf8') as f_input:
#     for row in csv.DictReader(f_input, delimiter='|', fieldnames=csv_header[:-1], restkey=csv_header[-1], skipinitialspace=True):
#         try:
#             temp_title = join_txt(row['title'])
#             rows.append([row['tweetid'], row['date'], temp_title,' '.join(row['what'])])
#         except KeyError:
# #             print("Hello")
#             rows.append([row['tweetid'], row['date'], temp_title,' '])

# frame = pd.DataFrame(rows)
# print(frame)

In [9]:
# frame.columns = frame_header
frame.head()

,tweetid,date,title,url
0,576880531301801984,Sat Mar 14 23:00:11 +0000 2015,An abundance of online info can turn us into e...,http://cnn.it/1L1t1Fv
1,576820122666471424,Sat Mar 14 19:00:08 +0000 2015,A plant-based diet that incorporates fish may ...,http://cnn.it/1xdpsjT
2,576744652717461504,Sat Mar 14 14:00:15 +0000 2015,It doesn't take much to damage your hearing at...,http://cnn.it/1BOphBk
3,576736754436304896,Sat Mar 14 13:28:52 +0000 2015,RT @CNN: Forever young? Discover this island’s...,http://cnn.it/1GNdmqc
4,576736614766010368,Sat Mar 14 13:28:18 +0000 2015,RT @CNN: Is post-traumatic stress disorder in ...,http://cnn.it/1xls8w5


In [10]:
frame['title'][123]

'Frozen mania: the scientific reason behind why kids can\'t "Let it go" &amp; why that may be ok  '

In [11]:
# Scrapt form web
df_temp = pd.DataFrame()
frame_urls = frame['url'].values

In [12]:
# -------------------------------------------------
# Run for this jupyter notebook
title_attempt = True
for key, url in enumerate(frame_urls):
    try:
        # Start the connection
        res = requests.get(url)        

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
            print(url)
            
            # Initialize XPath
            xpath_selector = Selector(text=res.text)
            
            list_content = dict()
            
            # Scrapt Text with XPath
            contents = xpath_selector.xpath('//*[@class="zn-body__paragraph"]/text()').extract()
            contents = join_txt(contents)
            print(len(contents))
#             # Tags
#             tags = xpath_selector.xpath('').extract()     
#             print(len(tags))
            # Author
            author = xpath_selector.xpath('//span[@class="metadata__byline__author"]/a/text()').extract()
            if len(author):
                author = author[0]
            else:
                author = ''
            print(len(author))
            list_content.update({'contents': contents,'tags': '','author': author})
            
            if title_attempt:
                print("TItle: ", title_attempt)
                df_temp = pd.DataFrame(columns = list(list_content.keys()))
                frame = pd.concat([frame, df_temp])
                print(frame.columns)                
                title_attempt = False
                
            for k_, v_ in list_content.items():
                print(k_," : " ,len(v_))
                frame.loc[key,k_] = v_
            print("\n")

        else:
            print("['ERRORS'] {} , at {}".format(res.status_code, url))
    except:
        pass
    

Header:   200
http://cnn.it/1L1t1Fv
6411
0
TItle:  True
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')
contents  :  6411
tags  :  0
author  :  0




scripts.py:38: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Find all possible hastags


Header:   200
http://cnn.it/1xdpsjT
1309
0
contents  :  1309
tags  :  0
author  :  0


Header:   200
http://cnn.it/1BOphBk
3634
0
contents  :  3634
tags  :  0
author  :  0


Header:   200
http://cnn.it/1GNdmqc
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1xls8w5
3646
0
contents  :  3646
tags  :  0
author  :  0


Header:   200
http://cnn.it/1GNiH0L
3214
0
contents  :  3214
tags  :  0
author  :  0


Header:   200
http://cnn.it/1AklXJQ
4457
0
contents  :  4457
tags  :  0
author  :  0


Header:   200
http://cnn.it/1AklXJQ
4457
0
contents  :  4457
tags  :  0
author  :  0


Header:   200
http://cnn.it/1GNhBC8
2866
0
contents  :  2866
tags  :  0
author  :  0


Header:   200
http://cnn.it/18HDKDg
1989
8
contents  :  1989
tags  :  0
author  :  8


Header:   200
http://cnn.it/1EhlrjJ
6890
0
contents  :  6890
tags  :  0
author  :  0


Header:   200
http://cnn.it/1AukT5d
2505
13
contents  :  2505
tags  :  0
author  :  13


Header:   200
http://cnn.it/1x1ffMU
1309
0
cont

Header:   200
http://cnn.it/1FbGQdQ
4209
0
contents  :  4209
tags  :  0
author  :  0


Header:   200
http://cnn.it/1xu62WP
5041
13
contents  :  5041
tags  :  0
author  :  13


Header:   200
http://cnn.it/1C37Knn
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1zG1fYh
3004
16
contents  :  3004
tags  :  0
author  :  16


Header:   200
http://cnn.it/1uUh47Z
4183
17
contents  :  4183
tags  :  0
author  :  17


Header:   200
http://cnn.it/1zG1fYh
3004
16
contents  :  3004
tags  :  0
author  :  16


Header:   200
http://cnn.it/1C0tErm
5462
13
contents  :  5462
tags  :  0
author  :  13


Header:   200
http://cnn.it/1zG1fYh
3004
16
contents  :  3004
tags  :  0
author  :  16


Header:   200
http://cnn.it/1zG1fYh
3004
16
contents  :  3004
tags  :  0
author  :  16


Header:   200
http://cnn.it/18KycIH
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1zG1fYh
3004
16
contents  :  3004
tags  :  0
author  :  16


Header:   200
http://cnn.it/1zG1fYh
300

Header:   200
http://cnn.it/1sKcma1
3079
0
contents  :  3079
tags  :  0
author  :  0


Header:   200
http://cnn.it/1xEppyc
8803
0
contents  :  8803
tags  :  0
author  :  0


Header:   200
http://cnn.it/ZY7U11
5539
0
contents  :  5539
tags  :  0
author  :  0


['ERRORS'] 404 , at http://t.co/cI
Header:   200
http://cnn.it/ZY7U11
5539
0
contents  :  5539
tags  :  0
author  :  0


Header:   200
http://youtu.be/gsTtJvvpo3U?list
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1pXE17J
5320
0
contents  :  5320
tags  :  0
author  :  0


Header:   200
http://cnn.it/1tCOLxj
2319
19
contents  :  2319
tags  :  0
author  :  19


Header:   200
http://cnn.it/1tBpLpY
3231
0
contents  :  3231
tags  :  0
author  :  0


Header:   200
http://bit.ly/130s5No
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1tDDJJe
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://cnn.it/1yktp7Q
0
0
contents  :  0
tags  :  0
author  :  0


Header:   200
http://

...

Header:   200
http://www.cnn.com/2013/01/29/health/steve-jobs-all-fruit-diet/index.html
1757
0
contents  :  1757
tags  :  0
author  :  0


['ERRORS'] 500 , at http://at.cnn.com/4TMsCSG
['ERRORS'] 500 , at http://at.cnn.com/V5LGknQ...
['ERRORS'] 500 , at http://at.cnn.com/VSovKxm
['ERRORS'] 500 , at http://at.cnn.com/e61auCj
['ERRORS'] 500 , at http://at.cnn.com/yFwAO6M
['ERRORS'] 500 , at http://at.cnn.com/qmCOHwH
['ERRORS'] 500 , at http://at.cnn.com/00FvvDW
['ERRORS'] 500 , at http://at.cnn.com/Loynle9
['ERRORS'] 500 , at http://at.cnn.com/YQV5den
['ERRORS'] 500 , at http://at.cnn.com/Wduov8L
['ERRORS'] 500 , at http://at.cnn.com/wQNHyvS
['ERRORS'] 500 , at http://at.cnn.com/65QFCV1
['ERRORS'] 500 , at http://at.cnn.com/ldOnRkn
['ERRORS'] 500 , at http://at.cnn.com/05DB8eK
['ERRORS'] 500 , at http://at.cnn.com/KTWkjuX
['ERRORS'] 500 , at http://at.cnn.com/aoEme5t
['ERRORS'] 500 , at http://at.cnn.com/PsGhbK8
['ERRORS'] 500 , at http://at.cnn.com/4Q740f7
['ERRORS'] 500 , at http://at.

['ERRORS'] 500 , at http://at.cnn.com/fdC8HDA
['ERRORS'] 500 , at http://at.cnn.com/uhedgvJ
['ERRORS'] 500 , at http://at.cnn.com/MJo2AP7
['ERRORS'] 500 , at http://at.cnn.com/zwoEQR6
['ERRORS'] 500 , at http://at.cnn.com/dn6dMDJ
['ERRORS'] 500 , at http://at.cnn.com/c1FAR6k
['ERRORS'] 500 , at http://at.cnn.com/NcoRAjp
['ERRORS'] 500 , at http://at.cnn.com/cGvyycI
['ERRORS'] 500 , at http://at.cnn.com/39avIHe
['ERRORS'] 500 , at http://at.cnn.com/qyXtLvf
['ERRORS'] 500 , at http://at.cnn.com/3Kc5YZR
['ERRORS'] 500 , at http://at.cnn.com/Wc9b10l
['ERRORS'] 500 , at http://at.cnn.com/tyqj6xm
['ERRORS'] 500 , at http://at.cnn.com/KdThoOZ
['ERRORS'] 500 , at http://at.cnn.com/JpRkFHX
['ERRORS'] 500 , at http://at.cnn.com/TtUGD54
['ERRORS'] 500 , at http://at.cnn.com/OrxGoyt
['ERRORS'] 500 , at http://at.cnn.com/Li99Oh6
['ERRORS'] 500 , at http://at.cnn.com/RSoCfRt
['ERRORS'] 500 , at http://at.cnn.com/AEjV3nh
['ERRORS'] 500 , at http://at.cnn.com/uFJ3LnD
['ERRORS'] 500 , at http://at.cnn.

['ERRORS'] 500 , at http://at.cnn.com/QUhado
['ERRORS'] 500 , at http://at.cnn.com/qbctKx
['ERRORS'] 500 , at http://at.cnn.com/a738kx
['ERRORS'] 500 , at http://at.cnn.com/dGTWAo
['ERRORS'] 500 , at http://at.cnn.com/uRaKed
['ERRORS'] 500 , at http://at.cnn.com/vrJJCl
['ERRORS'] 500 , at http://at.cnn.com/lrNIPt
['ERRORS'] 500 , at http://at.cnn.com/cQlzg6
['ERRORS'] 500 , at http://at.cnn.com/Do22os
['ERRORS'] 500 , at http://at.cnn.com/TeHGDN
['ERRORS'] 500 , at http://at.cnn.com/CsCyHC
['ERRORS'] 500 , at http://at.cnn.com/a9zTQ8
['ERRORS'] 500 , at http://at.cnn.com/funHCA
['ERRORS'] 500 , at http://at.cnn.com/KfzMel
['ERRORS'] 500 , at http://at.cnn.com/ZD91P9
['ERRORS'] 500 , at http://at.cnn.com/IgNVOg
['ERRORS'] 500 , at http://at.cnn.com/r1eERV
['ERRORS'] 500 , at http://at.cnn.com/hkM9zk
['ERRORS'] 500 , at http://at.cnn.com/gHlFF1
['ERRORS'] 500 , at http://at.cnn.com/TbcNSj
['ERRORS'] 500 , at http://at.cnn.com/9qobtI
['ERRORS'] 500 , at http://at.cnn.com/LBhLvW
['ERRORS']

['ERRORS'] 500 , at http://at.cnn.com/gFIhPQ
['ERRORS'] 500 , at http://at.cnn.com/y3G1Rr
['ERRORS'] 500 , at http://at.cnn.com/cef1zJ
['ERRORS'] 500 , at http://at.cnn.com/2GzjFh
['ERRORS'] 500 , at http://at.cnn.com/sGXKSS
['ERRORS'] 500 , at http://at.cnn.com/djr3JC
['ERRORS'] 500 , at http://at.cnn.com/mpgGgL
['ERRORS'] 500 , at http://at.cnn.com/yU3mCN
['ERRORS'] 500 , at http://at.cnn.com/uC0two
Header:   200
http://bit.ly/SgeV7L
0
0
contents  :  0
tags  :  0
author  :  0


['ERRORS'] 500 , at http://at.cnn.com/1EwI8h
['ERRORS'] 500 , at http://at.cnn.com/cCXCRV
['ERRORS'] 500 , at http://at.cnn.com/io3GMz
['ERRORS'] 500 , at http://at.cnn.com/QumYav
['ERRORS'] 500 , at http://at.cnn.com/ZwwTu7
['ERRORS'] 500 , at http://at.cnn.com/c9UvAW
['ERRORS'] 500 , at http://at.cnn.com/61a3oy
['ERRORS'] 500 , at http://at.cnn.com/ejTnAw
['ERRORS'] 500 , at http://at.cnn.com/Z1uivL
['ERRORS'] 500 , at http://at.cnn.com/MdAS4p
['ERRORS'] 500 , at http://at.cnn.com/1oehFX
['ERRORS'] 500 , at 

['ERRORS'] 500 , at http://at.cnn.com/hBvh0AS
['ERRORS'] 500 , at http://at.cnn.com/oKL2aDq
['ERRORS'] 500 , at http://at.cnn.com/WhljrzD
['ERRORS'] 500 , at http://at.cnn.com/THd49GI
['ERRORS'] 500 , at http://at.cnn.com/mKqmJtP
['ERRORS'] 500 , at http://at.cnn.com/9S9QfuK
['ERRORS'] 500 , at http://at.cnn.com/R9FPCAI
['ERRORS'] 500 , at http://at.cnn.com/dhSe1Hx
['ERRORS'] 500 , at http://at.cnn.com/WzkgFSB
['ERRORS'] 500 , at http://at.cnn.com/nUmkD2j
['ERRORS'] 500 , at http://at.cnn.com/hPAakcS
['ERRORS'] 500 , at http://at.cnn.com/TsRAN5M
['ERRORS'] 500 , at http://at.cnn.com/nMNINqm
['ERRORS'] 500 , at http://at.cnn.com/FdtPfx4
['ERRORS'] 500 , at http://at.cnn.com/AvOsCsF
['ERRORS'] 500 , at http://at.cnn.com/0EViXX8
['ERRORS'] 500 , at http://at.cnn.com/Kt8EaBb
['ERRORS'] 500 , at http://at.cnn.com/0Wh892Y
['ERRORS'] 500 , at http://at.cnn.com/UH47hvz
['ERRORS'] 500 , at http://at.cnn.com/wvUWwFD
['ERRORS'] 500 , at http://at.cnn.com/jWxeA5C
['ERRORS'] 500 , at http://at.cnn.

In [13]:
frame.shape

(4057, 7)

In [14]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4057 entries, 0 to 4056
Data columns (total 7 columns):
author      1537 non-null object
contents    1537 non-null object
date        4057 non-null object
tags        1537 non-null object
title       4057 non-null object
tweetid     4057 non-null float64
url         4057 non-null object
dtypes: float64(1), object(6)
memory usage: 413.6+ KB


In [15]:
frame.head()

,author,contents,date,tags,title,tweetid,url
0,,"Although we all laughed at the time, it was wi...",Sat Mar 14 23:00:11 +0000 2015,,An abundance of online info can turn us into e...,5.768805e+17,http://cnn.it/1L1t1Fv
1,,"Yet , from last year found an average weight ...",Sat Mar 14 19:00:08 +0000 2015,,A plant-based diet that incorporates fish may ...,5.768201e+17,http://cnn.it/1xdpsjT
2,,It's not just old folks who suffer hearing los...,Sat Mar 14 14:00:15 +0000 2015,,It doesn't take much to damage your hearing at...,5.767447e+17,http://cnn.it/1BOphBk
3,,,Sat Mar 14 13:28:52 +0000 2015,,RT @CNN: Forever young? Discover this island’s...,5.767368e+17,http://cnn.it/1GNdmqc
4,,"tend to cluster into three areas: , • Relivin...",Sat Mar 14 13:28:18 +0000 2015,,RT @CNN: Is post-traumatic stress disorder in ...,5.767366e+17,http://cnn.it/1xls8w5


In [16]:
frame.to_csv('./run_data/cnn_scrapt_v2.csv', sep='|', encoding='utf-8', index=False)

In [17]:
### Regex
# /div[@class="zn-body__read-all"]/div/text()

# --> Selenium Support